In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score

DATA PREPROCCESSING

In [20]:
#Baca Dataframe
#Data Train
df = pd.read_csv('survey lung cancer.csv')
#Data Test
data_test = pd.read_excel('Book2.xlsx')


In [3]:
#Mengambil tabel kelas/class LUNG_CANCER lalu di replace dengan tipe int
#Menyimpan class baru dengan tipe int ke variabel class_df
class_df = df.loc[:, ['LUNG_CANCER']]
cleanup_nums= {"LUNG_CANCER": {"NO":0, "YES":1}}
class_df = class_df.replace(cleanup_nums)


In [4]:
#Delete table LUNG_CANCER pada dataframe yang berjenis object
newDF = df.drop(["LUNG_CANCER"], axis=1)


In [5]:
#gabungkan tabel yang tipenya sudah dirubah dengan dataset
frame= [class_df, newDF ]
complete_df = pd.concat(frame, axis=1, join='inner')
complete_df.head()

,LUNG_CANCER,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN
0,1,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2
1,1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2
2,0,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2
3,0,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2
4,0,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1


In [6]:
#Mencari missing value dan hapus baris jika ada missing value

complete_df.dropna(inplace=True)
print(complete_df.isna().sum())
print(complete_df.shape)

LUNG_CANCER              0
GENDER                   0
AGE                      0
SMOKING                  0
YELLOW_FINGERS           0
ANXIETY                  0
PEER_PRESSURE            0
CHRONIC DISEASE          0
FATIGUE                  0
ALLERGY                  0
WHEEZING                 0
ALCOHOL CONSUMING        0
COUGHING                 0
SHORTNESS OF BREATH      0
SWALLOWING DIFFICULTY    0
CHEST PAIN               0
dtype: int64
(309, 16)


In [7]:
#membuat tabel gender dari object menjadi int dengan one hot encoding
#Mengapa tidak sekalian dengan class LUNG_CANCER nya? 
#karena jika menggunakan one hot encoding akan menjadi multiclass sehingga idak bisa melakukan knn
df_ohe = pd.get_dummies(complete_df)
df_ohe.shape

(309, 17)

In [8]:
#Setelah di preprocess data train akan diambil 10 teraatas agar tidak terlalu banyak
df_ohe = df_ohe.head(10)
df_ohe

,LUNG_CANCER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,GENDER_F,GENDER_M
0,1,69,1,2,2,1,1,2,1,2,2,2,2,2,2,0,1
1,1,74,2,1,1,1,2,2,2,1,1,1,2,2,2,0,1
2,0,59,1,1,1,2,1,2,1,2,1,2,2,1,2,1,0
3,0,63,2,2,2,1,1,1,1,1,2,1,1,2,2,0,1
4,0,63,1,2,1,1,1,1,1,2,1,2,2,1,1,1,0
5,1,75,1,2,1,1,2,2,2,2,1,2,2,1,1,1,0
6,1,52,2,1,1,1,1,2,1,2,2,2,2,1,2,0,1
7,1,51,2,2,2,2,1,2,2,1,1,1,2,2,1,1,0
8,0,68,2,1,2,1,1,2,1,1,1,1,1,1,1,1,0
9,1,53,2,2,2,2,2,1,2,1,2,1,1,2,2,0,1


In [9]:
#Membuat array features dari dataset
features = [col for col in df_ohe.columns]
features.remove("LUNG_CANCER")
#Memisahkan Kelas dan Fitur dari data test dan data training
X_train = np.array(df_ohe[features])
Y_train = np.array(df_ohe['LUNG_CANCER'])
X_test =  np.array(data_test[features])
Y_test =  np.array(data_test['LUNG_CANCER'])

KLASIFIKASI DENGAN KNN

In [14]:
#klasifikasi menggunakan knn dengan neighbors 3, dan distance
knn = KNeighborsClassifier(
    n_neighbors=3,
    weights='distance',
)
knn.fit(X_train, Y_train)
X_test[0]

array([78,  2,  2,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0,  1],
      dtype=int64)

In [12]:
# Prediksi kelas data test yang belum diketahui
predict = knn.predict(X_test)
predict

array([1, 1, 0], dtype=int64)

<b>MENGHITUNG DISTANCE TIAP DATA TEST DARI DATA TRAIN<b>

In [18]:
from scipy.spatial.distance import euclidean
#Jarak Data Test 1
jarak_data_test1 = [euclidean(X_test[0], d) for d in X_train]
jarak_data_test1

[9.539392014169456,
 4.898979485566356,
 19.235384061671343,
 15.165750888103101,
 15.231546211727817,
 4.358898943540674,
 26.153393661244042,
 27.129319932501073,
 10.295630140987,
 25.11971337416094]

In [17]:
#Jarak Data Test 2
jarak_data_test2 = [euclidean(X_test[1], d) for d in X_train]
jarak_data_test2

[5.656854249492381,
 2.6457513110645907,
 15.132745950421556,
 11.532562594670797,
 11.357816691600547,
 2.449489742783178,
 22.15851980616034,
 23.173260452512935,
 6.855654600401044,
 21.213203435596427]

In [19]:
#Jarak Data Test 3
jarak_data_test3 = [euclidean(X_test[2], d) for d in X_train]
jarak_data_test3

[10.392304845413264,
 15.329709716755891,
 1.7320508075688772,
 5.196152422706632,
 4.358898943540674,
 16.1245154965971,
 7.416198487095663,
 8.54400374531753,
 9.219544457292887,
 7.0710678118654755]